In [ ]:
#Load packages
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import os
import warnings
import functools
import seaborn as sns
import scipy.stats
import anndata
#import dandelion as ddl
import tables
from rpy2.robjects import r

In [ ]:
from collections import defaultdict

In [ ]:
#Create blank dictionaries 
adata_tmp = defaultdict(dict)
adata_tmp2 = defaultdict(dict)

In [ ]:
#Change working directory 
os.chdir("/home/jovyan/data/ClatCov/")


In [ ]:
#Load in metadata
sampleList = pd.read_csv('/Users/matthewcoates/Documents/Cambridge/COVID_NASAL_test_meta_cellbender.csv')
sampleList

In [ ]:
#Check 
sampleList.sampleid

In [ ]:
#Create list of samples 
samplelist = sampleList['sampleid'].tolist()
samplelist

In [ ]:
#Ben's code for processing cellbender data 
"Functions for downstream work with outputs of remove-background."""

import tables
import numpy as np
import scipy.sparse as sp
from typing import Dict


def dict_from_h5(file: str) -> Dict[str, np.ndarray]:
    """Read in everything from an h5 file and put into a dictionary."""
    d = {}
    with tables.open_file(file) as f:
        # read in everything
        for array in f.walk_nodes("/", "Array"):
            d[array.name] = array.read()
    return d


def anndata_from_h5(file: str,
                    analyzed_barcodes_only: bool = True) -> 'anndata.AnnData':
    """Load an output h5 file into an AnnData object for downstream work.

    Args:
        file: The h5 file
        analyzed_barcodes_only: False to load all barcodes, so that the size of
            the AnnData object will match the size of the input raw count matrix.
            True to load a limited set of barcodes: only those analyzed by the
            algorithm. This allows relevant latent variables to be loaded
            properly into adata.obs and adata.obsm, rather than adata.uns.

    Returns:
        adata: The anndata object, populated with inferred latent variables
            and metadata.

    """

    try:
        import anndata
    except ImportError:
        raise ImportError('The anndata package must be installed to use the '
                          'function anndata_from_h5()')

    d = dict_from_h5(file)
    X = sp.csc_matrix((d.pop('data'), d.pop('indices'), d.pop('indptr')),
                      shape=d.pop('shape')).transpose().tocsr()

    if analyzed_barcodes_only:
        if 'barcodes_analyzed_inds' in d.keys():
            X = X[d['barcodes_analyzed_inds'], :]
            d['barcodes'] = d['barcodes'][d['barcodes_analyzed_inds']]
        elif 'barcode_indices_for_latents' in d.keys():
            X = X[d['barcode_indices_for_latents'], :]
            d['barcodes'] = d['barcodes'][d['barcode_indices_for_latents']]
        else:
            print('Warning: analyzed_barcodes_only=True, but the key '
                  '"barcodes_analyzed_inds" or "barcode_indices_for_latents" '
                  'is missing from the h5 file. '
                  'Will output all barcodes, and proceed as if '
                  'analyzed_barcodes_only=False')

    # Construct the count matrix.
    adata = anndata.AnnData(X=X,
                            obs={'barcode': d.pop('barcodes').astype(str)},
                            var={'gene_name': (d.pop('gene_names') if 'gene_names' in d.keys()
                                               else d.pop('name')).astype(str)})
    adata.obs.set_index('barcode', inplace=True)
    adata.var.set_index('gene_name', inplace=True)

    # Add other information to the adata object in the appropriate slot.
    for key, value in d.items():
        try:
            value = np.asarray(value)
            if len(value.shape) == 0:
                adata.uns[key] = value
            elif value.shape[0] == X.shape[0]:
                if (len(value.shape) < 2) or (value.shape[1] < 2):
                    adata.obs[key] = value
                else:
                    adata.obsm[key] = value
            elif value.shape[0] == X.shape[1]:
                if value.dtype.name.startswith('bytes'):
                    adata.var[key] = value.astype(str)
                else:
                    adata.var[key] = value
            else:
                adata.uns[key] = value
        except Exception:
            print('Unable to load data into AnnData: ', key, value, type(value))

    if analyzed_barcodes_only:
        for col in adata.obs.columns[adata.obs.columns.str.startswith('barcodes_analyzed')
                                     | adata.obs.columns.str.startswith('barcode_indices')]:
            try:
                del adata.obs[col]
            except Exception:
                pass

    return adata


def load_anndata_from_input_and_output(input_file: str,
                                       output_file: str,
                                       analyzed_barcodes_only: bool = True,
                                       input_layer_key: str = 'cellranger') -> 'anndata.AnnData':
    """Load remove-background output count matrix into an anndata object,
    together with remove-background metadata and the raw input counts.

    Args:
        input_file: Raw h5 file used as input for remove-background.
        output_file: Output h5 file created by remove-background (can be
            filtered or not).
        analyzed_barcodes_only: Argument passed to anndata_from_h5().
            False to load all barcodes, so that the size of
            the AnnData object will match the size of the input raw count matrix.
            True to load a limited set of barcodes: only those analyzed by the
            algorithm. This allows relevant latent variables to be loaded
            properly into adata.obs and adata.obsm, rather than adata.uns.
        input_layer_key: Key of the anndata.layer that is created for the raw
            input count matrix.

    Return:
        adata_out: AnnData object with counts before and after remove-background,
            as well as inferred latent variables from remove-background.

    """

    # Load input data.
    adata_raw = anndata_from_h5(input_file, analyzed_barcodes_only=False)

    # Load remove-background output data.
    adata_out = anndata_from_h5(output_file, analyzed_barcodes_only=analyzed_barcodes_only)

    # Subset the raw dataset to the relevant barcodes.
    adata_raw = adata_raw[adata_out.obs.index]

    # Put count matrices into 'layers' in anndata for clarity.
    adata_out.layers[input_layer_key] = adata_raw.X.copy()
    adata_out.layers['cellbender'] = adata_out.X.copy()

    # Pre-compute a bit of metadata.
    adata_out.var['n_cellranger'] = np.array(adata_out.layers['cellranger'].sum(axis=0)).squeeze()
    adata_out.var['n_cellbender'] = np.array(adata_out.layers['cellbender'].sum(axis=0)).squeeze()

    return adata_out



In [ ]:
samplelist

In [ ]:
#adata_cellbender = defaultdict(dict)

In [ ]:

#adata_cellbender = load_anndata_from_input_and_output(input_file = ("COVID_combined_trans/Sample_Fq21/raw_feature_bc_matrix.h5"),
                                       #output_file = ("COVID_combined_trans/Sample_Fq21/cellbender_out/output.h5"))
#adata_cellbender
#Note: adata.obsm['latent_gene_encoding'] is the same as z / adata.obsm['X_cellbender'] from the cellbender vignette

In [ ]:
#Create blank dictionaries for cellbender file path slots 
input_cfile = defaultdict(dict)
output_cfile = defaultdict(dict)
input_newfile = defaultdict(dict)

In [ ]:
input_cfile

In [ ]:
#Load in appropriate file paths for each sample 
for x in sampleList.sampleid:
    input_cfile[x] = ("COVID_IRVAS_combined_trans/" +x+ "/raw_feature_bc_matrix.h5")
    output_cfile[x] = ("COVID_IRVAS_combined_trans/" +x+ "/cellbender_out/output.h5")
    input_newfile[x] = ("COVID_IRVAS_combined_trans/" +x+ "/cellbender_out/output_filtered.h5")
input_newfile["Sample_Fq21"]

In [ ]:
#Create blank dictionary for actual data
adata_cellbender = defaultdict(dict)

In [ ]:
#Load in cellbender processed data 
for x in samplelist:
    adata_cellbender[x] = load_anndata_from_input_and_output(input_file = input_cfile[x], output_file = output_cfile[x])
adata_cellbender

In [ ]:
#Check 
adata_cellbender['Sample_Fq21']

In [ ]:
#Check 
adata_cellbender['Sample_Fq_18'].var

In [ ]:
#Run scrublet for auto doublet detection 
for x in samplelist:
    sc.external.pp.scrublet(adata_cellbender[x], expected_doublet_rate = 0.06)
adata_cellbender[x]

In [ ]:
#Make name simpler
adata = adata_cellbender
adata

In [ ]:
#Check 
adata['Sample_Fq21'].obs

In [ ]:
#Change name of meta to correspond with code below 
sampleInfo = sampleList
sampleInfo

In [ ]:
#Load in sample metadata
for i in sampleInfo.index:
    adata[sampleInfo['sampleid'].at[i]].obs['Sampleid'] = sampleInfo['sampleid'].at[i] 
    adata[sampleInfo['sampleid'].at[i]].obs['Patient'] = sampleInfo['study_id/patient_id'].at[i]    
    adata[sampleInfo['sampleid'].at[i]].obs['Sampletype'] = sampleInfo['site'].at[i]
    adata[sampleInfo['sampleid'].at[i]].obs['Age_group'] = sampleInfo['age'].at[i]
    adata[sampleInfo['sampleid'].at[i]].obs['Gender'] = sampleInfo['Gender'].at[i]
    adata[sampleInfo['sampleid'].at[i]].obs['Cohort'] = sampleInfo['cohort'].at[i]
    adata[sampleInfo['sampleid'].at[i]].obs['Diseasetype'] = sampleInfo['Sub-cohort'].at[i]
    adata[sampleInfo['sampleid'].at[i]].obs['Batch'] = sampleInfo['batch'].at[i]
    adata[sampleInfo['sampleid'].at[i]].obs['Study'] = sampleInfo['cohort.1'].at[i]
    adata[sampleInfo['sampleid'].at[i]].obs['Nasal_inflammation'] = sampleInfo['Nasal inflammation'].at[i]
    adata[sampleInfo['sampleid'].at[i]].obs['COVID_severity'] = sampleInfo['COVID Severity'].at[i]
adata

In [ ]:
#Remove these variables as they are causing issues with concatenation. 
for x in samplelist:
    del adata[x].var['n_cellranger']
    del adata[x].var['n_cellbender']
    del adata[x].var['ambient_expression']
    
adata

In [ ]:
for x in samplelist:
    adata[x].obs_names_make_unique()
adata

In [ ]:
for x in samplelist:
    adata[x].var_names_make_unique()
adata

In [ ]:
adata_list = []
adata_list

In [ ]:
for x in samplelist:
    adata_list.append(adata[x], )
adata_list

In [ ]:
adata = adata_list[0].concatenate(adata_list[1:], batch_key = 'sampleid')
adata

In [ ]:
adata.obs

In [ ]:
adata.write('COVID_combined_trans/out/COV_ctrl_cellbender_raw_nopp_250621.h5ad')

In [ ]:
adata2 = sc.read_h5ad("COVID_combined_trans/out/COV_ctrl_missing_Soupx_raw_nopp_250621.h5ad")
adata2

In [ ]:
adata.obs['soup_correction'] = 'Cellbender'
adata.obs

In [ ]:
adata2.obs['soup_correction'] = 'SoupX'
adata2.obs

In [ ]:
#Concatenate 
adata_concat = adata.concatenate(adata2, batch_key = 'sampleid')
adata_concat

In [ ]:
adata_concat.write("COVID_combined_trans/out/adata_cellbender+soupxmissingdata_raw_nopp_250621.h5ad")

In [ ]:
#Check object
adata_concat

In [ ]:
#What are duplicate var values? 
adata_concat.var

In [ ]:
#Condense duplicate var values in to one. Delete rest (done above already)
adata_concat.var['gene_ids'] = adata_concat.var['id-0']
adata_concat.var['feature_types'] = adata_concat.var['feature_type-0']
adata_concat.var['genome'] = adata_concat.var['genome-0']
adata_concat.var.drop(columns = ['feature_type-0', 'genome-0', 'id-0', 'ambient_expression-0-0', 'n_cellranger-0-0', 'n_cellbender-0-0', 'ambient_expression-1-0', 'n_cellranger-1-0', 'n_cellbender-1-0', 'ambient_expression-10-0', 'n_cellranger-10-0', 'n_cellbender-10-0', 'ambient_expression-11-0', 'n_cellranger-11-0', 'n_cellbender-11-0', 'ambient_expression-12-0', 'n_cellranger-12-0', 'n_cellbender-12-0', 'ambient_expression-13-0', 'n_cellranger-13-0', 'n_cellbender-13-0', 'ambient_expression-14-0', 'n_cellranger-14-0', 'n_cellbender-14-0', 'ambient_expression-15-0', 'n_cellranger-15-0', 'n_cellbender-15-0', 'ambient_expression-16-0', 'n_cellranger-16-0', 'n_cellbender-16-0', 'ambient_expression-17-0', 'n_cellranger-17-0', 'n_cellbender-17-0', 'ambient_expression-18-0', 'n_cellranger-18-0', 'n_cellbender-18-0', 'ambient_expression-19-0', 'n_cellranger-19-0', 'n_cellbender-19-0', 'ambient_expression-2-0', 'n_cellranger-2-0', 'n_cellbender-2-0', 'ambient_expression-20-0', 'n_cellranger-20-0', 'n_cellbender-20-0', 'ambient_expression-21-0', 'n_cellranger-21-0', 'n_cellbender-21-0', 'ambient_expression-22-0', 'n_cellranger-22-0', 'n_cellbender-22-0', 'ambient_expression-23-0', 'n_cellranger-23-0', 'n_cellbender-23-0', 'ambient_expression-24-0', 'n_cellranger-24-0', 'n_cellbender-24-0', 'ambient_expression-25-0', 'n_cellranger-25-0', 'n_cellbender-25-0', 'ambient_expression-26-0', 'n_cellranger-26-0', 'n_cellbender-26-0', 'ambient_expression-27-0', 'n_cellranger-27-0', 'n_cellbender-27-0', 'ambient_expression-28-0', 'n_cellranger-28-0', 'n_cellbender-28-0', 'ambient_expression-29-0', 'n_cellranger-29-0', 'n_cellbender-29-0', 'ambient_expression-3-0', 'n_cellranger-3-0', 'n_cellbender-3-0', 'ambient_expression-30-0', 'n_cellranger-30-0', 'n_cellbender-30-0', 'ambient_expression-31-0', 'n_cellranger-31-0', 'n_cellbender-31-0', 'ambient_expression-32-0', 'n_cellranger-32-0', 'n_cellbender-32-0', 'ambient_expression-33-0', 'n_cellranger-33-0', 'n_cellbender-33-0', 'ambient_expression-34-0', 'n_cellranger-34-0', 'n_cellbender-34-0', 'ambient_expression-35-0', 'n_cellranger-35-0', 'n_cellbender-35-0', 'ambient_expression-36-0', 'n_cellranger-36-0', 'n_cellbender-36-0', 'ambient_expression-37-0', 'n_cellranger-37-0', 'n_cellbender-37-0', 'ambient_expression-38-0', 'n_cellranger-38-0', 'n_cellbender-38-0', 'ambient_expression-39-0', 'n_cellranger-39-0', 'n_cellbender-39-0', 'ambient_expression-4-0', 'n_cellranger-4-0', 'n_cellbender-4-0', 'ambient_expression-5-0', 'n_cellranger-5-0', 'n_cellbender-5-0', 'ambient_expression-6-0', 'n_cellranger-6-0', 'n_cellbender-6-0', 'ambient_expression-7-0', 'n_cellranger-7-0', 'n_cellbender-7-0', 'ambient_expression-8-0', 'n_cellranger-8-0', 'n_cellbender-8-0', 'ambient_expression-9-0', 'n_cellranger-9-0', 'n_cellbender-9-0', 'gene_ids-1', 'feature_types-1'], inplace = True)
adata_concat.var 

In [ ]:
#Check obs 
adata_concat.obs

In [ ]:
#Check object
adata_concat

In [ ]:
# How many doublets did scrublet pick up in cellbender object (named predicted doublet?
adata_concat.obs['predicted_doublet'].value_counts()

In [ ]:
#How many doublets in soupx object (names 'is_doublet')
adata_concat.obs['is_doublet'].value_counts()

In [ ]:
#Filter
sc.pp.filter_cells(adata_concat, min_genes=200)
sc.pp.filter_genes(adata_concat, min_cells=3)
adata_concat

In [ ]:
#Add in mitochondrial metadata 
adata_concat.var['mt'] = adata_concat.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_concat, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata_concat

In [ ]:
# violin plot of the computed quality measures.
sc.pl.violin(adata_concat, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter = 0.4, multi_panel = True)

In [ ]:
#Filter 
adata_concat = adata_concat[adata_concat.obs.n_genes_by_counts < 2500, :]
adata_concat = adata_concat[adata_concat.obs.pct_counts_mt < 15, :]
adata_concat

In [ ]:
# violin plot of the computed quality measures.
sc.pl.violin(adata_concat, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter = 0.4, multi_panel = True)

In [ ]:
# Remove the bad quality cells and continue
#adata = adata[adata.obs['filter_rna'] != "True"]
#adata
#I haven't done this because filter_rna scores were weird and getting rid of a lot of my cells. 

In [ ]:
# How many doublets did scrublet pick up once filtering done?
adata_concat.obs['predicted_doublet'].value_counts()

In [ ]:
# How many doublets did scrublet pick up once filtering done?
adata_concat.obs['is_doublet'].value_counts()

In [ ]:
#save unnormalised data
adata_concat.write('COVID_combined_trans/out/COVID_ctrl_raw__filtered_nolognorm_250621.h5ad')

In [ ]:
#relaod data as kernel died in normalising step
adata_concat = sc.read_h5ad('COVID_ctrl_raw_filtered_')
adata_concat

In [ ]:
# normalize to 10000 counts per cell
sc.pp.normalize_total(adata_concat, target_sum = 1e4)



In [ ]:
#Write out normalised but not log1p corrected data
adata_concat.write('COVID_combined_trans/out/COVID_ctrl_raw_filtered_norm_nolog_250621.h5ad')

In [ ]:
#lognormalise
sc.pp.log1p(adata_concat)

In [ ]:
#Write out lognormalised data
adata_concat.write('COVID_combined_trans/out/COVID_ctrl_raw_filtered_lognorm_250621.h5ad')

In [ ]:
# create a separate object to hold the raw counts and stash normalised data
adata_concat.raw = adata_concat
adata_concat_raw = adata_concat.copy()
# also do this
adata_concat.layers["counts"] = adata_concat.X.copy()
adata_concat_raw.layers["counts"] = adata_concat_raw.X.copy()

In [ ]:
adata_concat.raw

In [ ]:
adata_concat

In [ ]:
# How many doublets did scrublet pick up?
adata_concat.obs['predicted_doublet'].value_counts()

In [ ]:
# How many doublets did scrublet pick up?
adata_concat.obs['is_doublet'].value_counts()

In [ ]:
#Merge 'is_doublet' and 'predicted_doublet' from scrublet of the different combined objects
adata_concat.obs['combined_doublet'] = adata_concat.obs['predicted_doublet'].astype(str) + adata_concat.obs['is_doublet'].astype(str)

In [ ]:
#Check 
adata_concat.obs['combined_doublet'].value_counts()

In [ ]:
#Change terms so they are amenable to processing 
adata_concat.obs['combined_doublet'].replace('Falsenan','False',inplace=True)
adata_concat.obs['combined_doublet'].value_counts()

In [ ]:
#Change terms so they are amendable to processing. 
adata_concat.obs['combined_doublet'].replace('nanFalse','False',inplace=True)
adata_concat.obs['combined_doublet'].value_counts()

In [ ]:
#Remove the scrublet doublets
adata_concat = adata_concat[adata_concat.obs['combined_doublet'] == "False"]
adata_concat

In [ ]:
#Check 
adata_concat.obs['combined_doublet'].value_counts()

In [ ]:
# Identify highly-variable genes
sc.pp.highly_variable_genes(adata_concat, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
# plot highly_variable_genes
sc.pl.highly_variable_genes(adata_concat)

In [ ]:
## remove TRBV/TRAV and IGHV/IGLV/IGKV from the highly variable genes
#Decided not to remove TRGV/TRDV as need these for gdT subset identification. 
import re
for i in adata_concat.var.index:
    if re.search('^TR[AB]V|^IG[HKL]V', i):
        adata_concat.var.at[i, 'highly_variable'] = False
sc.pl.highly_variable_genes(adata_concat)

In [ ]:
# filter to only highly variable
adata_concat = adata_concat[:, adata_concat.var['highly_variable']]
adata_concat

In [ ]:
#Assign 'cellbender_cell' for latent_cell_probability >0.5 (not filtered on this as no latent cell probability for soupx cells, and filtering removed almost all cellbender low probability cells )
adata_concat.obs['cellbender_cell'] = adata_concat.obs['latent_cell_probability']>0.5
adata_concat

In [ ]:
adata_concat.obs['cellbender_cell'].value_counts()

In [ ]:
#Only 14 cells are latent_cell_prob <0.5 (in light of Soupx cells not having this value), therefore not filtered to this

In [ ]:
#Filter to remove cellbender assessed unlikely cells 
#adata = adata[adata.obs.cellbender_cell == True, :]
#adata

In [ ]:
#adata.obs['cellbender_cell'].value_counts()

In [ ]:
import multiprocessing
# regress and scale for PCA
sc.pp.regress_out(adata_concat, ['total_counts', 'pct_counts_mt'],n_jobs = multiprocessing.cpu_count()-4)


In [ ]:
sc.pp.scale(adata_concat, max_value = 10)


In [ ]:
# Principal component analysis
sc.tl.pca(adata_concat, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(adata_concat, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(adata_concat, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP on non-batch corrected data 
sc.tl.umap(adata_concat, n_components = 2, min_dist = 0.3)
sc.pl.umap(adata_concat, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
# run harmony for batch correction 
sc.external.pp.harmony_integrate(adata_concat, 'Patient')
'X_pca_harmony' in adata_concat.obsm


In [ ]:
#Check 
adata_concat

In [ ]:
# Compute the neighborhood graph for harmony corrected data. Seurat uses k = 20 as default
sc.pp.neighbors(adata_concat, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP for harmony corrected data. 
sc.tl.umap(adata_concat, n_components = 2, min_dist = 0.3)
sc.pl.umap(adata_concat, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
#Check 
adata_concat

In [ ]:
# find leiden clusters
sc.tl.leiden(adata_concat, resolution =0.5)


In [ ]:
#Check split amongst classifiers. Check if further splitting required. 
sc.pl.umap(adata_concat, color=['Sampleid','Patient', 'Sampletype', 'Gender', 'leiden', 'Cohort'], ncols = 3)

In [ ]:
#See how celltypist automated names match up. 
sc.pl.umap(adata_concat, color=['leiden', 'celltypist'], legend_loc = 'on data', ncols = 3)

In [ ]:
# create a palette for umap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
viridis = cm.get_cmap('viridis', 256)
newcolors = viridis(np.linspace(0, 1, 256))
grey = np.array([215/256, 215/256, 215/256, 1])
newcolors[:1, :] = grey
newcmp = ListedColormap(newcolors)

In [ ]:
# run marker gene test
sc.tl.rank_genes_groups(adata_concat, groupby = 'leiden', method = 'wilcoxon', n_genes = 30000)

In [ ]:
#Ploty dotplot of top 10 DEG by cluster. 
sc.pl.rank_genes_groups_dotplot(adata_concat, n_genes = 10, standard_scale = 'var', color_map = 'viridis')

In [ ]:
adata_concat

In [ ]:
#cluster marker genes from Chua et al. Nat Biotech 2020
Chuafeat = ['TP63', 'KRT5', 'S100A2', 'FABP5', 'SERPINB3', 'TMSB4X', 'IFIT1', 'IFIT2', 'IFITM3', 'ISG15', 'ISG20', 'OAS1', 'SCGB1A1', 'SCGB3A1', 'XBP1', 'VMO1', 'MUC5AC', 'PIGR', 'FOXN4', 'CCNO', 'MYCL', 'CDC20B', 'TUBA1B', 'PCM1', 'FOXJ1', 'EFHC1', 'CCDC153', 
              'CCDC113', 'MLF1', 'LZTFL1', 'FOXI1', 'CFTR', 'ASCL3', 'FOXI2', 'SFTPB', 'ANK2', 'SFTPB', 'ANK2', 'SPRR1A', 'SPRR2A', 'SPRR2D', 'SPRR2E', 'SPRR3', 'TMPRSS11E', 'IL1B', 'VCAN', 'CD14', 'CCL2', 'FCGR3A', 'CXCL10', 'IFIT1', 'CD68', 'FABP5', 'FCER1A', 'CD74', 'HLA-DQB1',
              'HLA-DRA', 'JCHAIN', 'APOE', 'NCAM1', 'HMGB2', 'STMN1', 'FOXP3', 'CTLA4', 'TNFRSF18', 'CD4', 'CD8B', 'CD8A', 'PRF1', 'GZMA', 'GZMB', 'GNLY', 'NKG7', 'CD3G', 'CD3E', 'KLRB1', 'IL32', 'S100A4', 'CD27', 'CD19', 'MS4A1', 'CD79A', 'IRF7', 'TLR7', 'CLEC4C', 'IL3RA', 'LYN', 
              'FCGR3B', 'ITGAX', 'HPGD', 'LTC4S', 'CPA3','CD69', 'ITGA1', 'KIT', 'HBB', 'PPBP']
Chuafeat

In [ ]:
#Marker genes from Travaglini 2020 Nature 
Trav_list = ['FOXJ1','KRT5','MUC5B',
   'MUC5B',
   'PRR4',
   'CFTR',
   'CALCA',
   'DCLK1',
   'AGER',
   'SFTPB',
   'ACKR1',
   'CA4',
   'PROX1',
   'CNN1',
   'CNN1',
   'COL1A1',
   'COL1A1',
   'COL1A1',
   'CSPG4',
   'MSLN',
   'SNAP25',
   'CD79A',
   'CD79A',
   'CD3E',
   'CD3E',
   'CD3E',
   'CD3E',
   'KLRD1',
   'CD3E',
   'S100A8',
   'MS4A2',
   'MS4A2',
   'SIGLEC8',
   'NRGN',
   'MARCO',
   'LILRB4',
   'CLEC9A',
   'CD14',
  'SCGB3A2',
   'TUBB1',
   'KRT14',
   'MUC5AC',
   'LPO',
   'FOXI1',
   'CHGA',
   'ASCL2',
   'PDPN',
   'BMX',
   'PDPN',
   'ACTA2',
   'PDGFRA',
   'TRPC6',
   'UPK3B',
   'CD24',
   'CD27',
   'CD8A',
   'CD4',
   'NKG7',
   'CD8B',
   'S100A9',
   'CPA3',
   'PPBP',
   'MSR1',
   'IRF8',
   'LAMP3',
   'CD1C',
   'S100A8',
   'TP73',
   'TP63',
   'SPDEF',
   'LTF',
   'ASCL3',
   'ASCL1',
   'CLIC5',
   'TAGLN',
   'ELN',
   'PLIN2',
   'PDGFRB',
   'MS4A1',
   'SLAMF7',
   'GZMK',
   'COTL1',
   'CCR7',
   'TYROBP',
   'FCER1G',
   'IFITM2',
   'TPSAB1',
   'PF4',
   'MRC1',
   'LILRA4',
   'PLD4',
   'FCGR3A',
   'CCDC78',
   'DAPL1',
   'MUC1',
   'RGS5',
   'DES',
   'ACTA2',
   'APOE',
   'CD19',
   'DUSP2',
   'GZMB',
   'LDHB',
   'LEF1',
   'TYROBP',
   'FCGR3B',
   'OST4',
   'ETV5', 
   'LGR6']
Trav_list

In [ ]:
#Plot Chua marker genes by cluster 
sc.pl.dotplot(adata_concat, Chuafeat, groupby = 'leiden')

In [ ]:
#Plot Travaglini genes by cluster. 
sc.pl.dotplot(adata_concat,Trav_list, groupby = 'leiden', title = 'COV_ctrl_wholeobject_Travaglini')

In [ ]:
#Plot ILC marker genes from Bjorkland 2016 Nature Immunology. 
ILC_Bjorkland = ['CCL3',
'CXCR3',
'IFNG',
'IL12RB1',
'TBX21',
'PTGDR2',
'IL17RB',
'IL1RL1',
'IL13',
'GATA3',
'NCR2',
'IL22',
'RORC',
'AHR',
'IL23R',
'IL1R1',
'EOMES',
'GZMA',
'GNLY',
'KLRC1']

In [ ]:
ILC_Bjorkland

In [ ]:
#Dotplot of Bjorkland ILC cluster genes. 
sc.pl.dotplot(adata_concat,ILC_Bjorkland, groupby = 'leiden', title = 'COV_ctrl_wholeobject_BjorklandILC')

In [ ]:
# give broad cluster annoations based on expression of top 10 DEG by cluster and marker gene expression from published datasets. 
new_dict = {'0':'CD4 T',
'1':'B',
'2':'MMP',
'3':'NK/NKT',
'4':'Plasma',
'5':'Epithelial',
'6':'Treg',
'7':'CD8 T',
'8':'B',
'9':'B',
'10':'Epithelial',
'11':'CD4 T',
'12':'CD4 T',
'13':'CD8 Tn',
'14':'CD4 T',
'15':'Erythrocytes',
'16':'Megakaryocyte',
'17':'CD4 T',
'18':'CD4 T',
'19':'CD4 T',
'20':'gdT',
'21':'B',
'22':'pDC',
'23':'B',
'24':'CD4 T',
'25':'Mast',
'26':'CD4 T',
'27':'B',
'28':'ILC',
'29':'Epithelial',
'30':'Epithelial'}
adata_concat.obs['broad_label'] = [new_dict[l] for l in adata_concat.obs['leiden']]
sc.pl.umap(adata_concat, color=['leiden', 'broad_label'], legend_fontoutline=2, legend_loc = 'on data')

In [ ]:

#Write object with broad annotations. 
adata_concat.write('COV_combined_broad_label_020721.h5ad', compression = 'gzip')
adata_concat.raw.to_adata().write('COV_combined_broad_label_raw.h5ad', compression = 'gzip')

In [ ]:
#reload as kernel died. 
adata_concat = sc.read_h5ad('COV_combined_broad_label_020721.h5ad')
adata_concat

In [ ]:
#Check percent_mito is defunct and pct_counts_mt should be used. 
adata_concat.obs['percent_mito']

In [ ]:
adata_concat.obs['pct_counts_mt']

In [ ]:
#percent_mito and n_counts are only for Soupx object, and have been calculated using scanpy
#calulate_qc_metrics de novo for all cells, so drop the redundant percent_mito and n_counts columns
#Use total_counts and pct_counts_mt instead
del adata_concat.obs['percent_mito']
del adata_concat.obs['n_counts']
adata_concat

In [ ]:
#Visualise 
sc.pl.umap(adata_concat, color=['broad_label','leiden', 'Sampletype'])

## Sub-clustering for fine cell type annotation

#First create a new column called `leiden_R_label` which I will update as I progress through.

In [ ]:
#Create new slot for labels in main object. 
adata_concat.obs['fine_label'] = adata_concat.obs['leiden']

In [ ]:
#Check 
adata_concat

In [ ]:
#Visualise
sc.pl.umap(adata_concat, color=['leiden', 'broad_label'], legend_fontoutline=2, legend_loc = 'on data')

### T cell

In [ ]:
# subset
rna_ = adata_concat[adata_concat.obs['leiden'].isin(['0','6','7','11','12','13','14','17','18','19','20','23','24','26','27','28'])]
rna_x = sc.AnnData(X = rna_.raw.X, obs = rna_.obs, var = rna_.raw.var)
rna_x.raw = rna_x
sc.pp.highly_variable_genes(rna_x, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(rna_x)
rna_x

In [ ]:
## remove TRBV/TRAV and IGHV/IGLV/IGKV from the highly variable genes
## allow for TRGV and TRDV so that i can do fine labelling for them
import re
for i in rna_x.var.index:
    if re.search('^TR[AB]V|^IG[HKL]V', i):
        rna_x.var.at[i, 'highly_variable'] = False
sc.pl.highly_variable_genes(rna_x)

In [ ]:
# subset to highly variable
rna_x = rna_x[:, rna_x.var['highly_variable']]


In [ ]:
# regress and scale for PCA
sc.pp.regress_out(rna_x, ['total_counts', 'pct_counts_mt'])


In [ ]:
sc.pp.scale(rna_x, max_value = 10)

In [ ]:
# Principal component analysis
sc.tl.pca(rna_x, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(rna_x, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# run harmony
sc.external.pp.harmony_integrate(rna_x, 'Patient')
'X_pca_harmony' in rna_x.obsm


In [ ]:
#Check 
rna_x

In [ ]:
# Compute the neighborhood graph correcting for batch using harmony PCA. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP - harmony corrected. 
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
#?What is the split by main object leiden cluster in reclustered object. 
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# find clusters in subsetted object. 
sc.tl.leiden(rna_x, resolution = 1, key_added = 'leiden_R')
sc.pl.umap(rna_x, color=['leiden', 'leiden_R'], size=6, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Where are key marker genes for t cell subsets expressed? 
sc.pl.umap(rna_x, color = ['TRDC', 'TRGV9', 'CD4', 'CD8B', 'FOXP3', 'CXCR5', 'PDCD1', 'CD8A','TRAV1-2', 'CD69', 'KLRB1','ITGAE'], size = 20)

In [ ]:
sc.pl.umap(rna_x, color = ['SELL', 'IFNG', 'PTGDR2','CCR3','CCR8','CXCR3','CCR5','IL13', 'IL5', 'IL4', 'IL17A', 'TRGC2', 'TRDV1', 'CD3E','TRGC1'], size = 20)

In [ ]:
# split specific clusters based on gene expression profiles
sc.tl.leiden(rna_x, resolution = .3, key_added = 'leiden_R2', restrict_to =('leiden_R', ['1']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split specific clusters
sc.tl.leiden(rna_x, resolution = .3, key_added = 'leiden_R2.1', restrict_to =('leiden_R2', ['2']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2.1'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split specific clusters
sc.tl.leiden(rna_x, resolution = .3, key_added = 'leiden_R2.2', restrict_to =('leiden_R2.1', ['3']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2.2'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split specific clusters
sc.tl.leiden(rna_x, resolution = .2, key_added = 'leiden_R2.3', restrict_to =('leiden_R2.2', ['7']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2.3'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Rank top 10 DEG by cluster and plot. 
sc.tl.rank_genes_groups(rna_x, groupby = 'leiden_R2.3', method = 'wilcoxon')
sc.tl.filter_rank_genes_groups(rna_x, min_fold_change=1)
sc.tl.dendrogram(rna_x, groupby = 'leiden_R2.3')
sc.pl.rank_genes_groups_dotplot(rna_x, n_genes = 10, standard_scale = 'var', color_map = 'viridis', key = 'rank_genes_groups_filtered')

In [ ]:
#Plot Chua 2020 nasal covid marker genes. 
sc.pl.dotplot(rna_x, Chuafeat, groupby = 'leiden_R2.3')

In [ ]:
#Plot travaglini 2020 lung marker genes. 
sc.pl.dotplot(rna_x, Trav_list, groupby = 'leiden_R2.3')

In [ ]:
#Plot Bjorkland ILC marker genes. 
sc.pl.dotplot(rna_x, ILC_Bjorkland, groupby = 'leiden_R2.3')

In [ ]:
Zhang2020_T = ['CD3E',
              'CD4',
              'CD8A',
              'CCR7',
              'SELL',
              'TCF7',
              'LEF1',
              'LTB',
              'S100A4',
              'GPR183',
              'CD69',
              'GZMK',
              'GZMA',
              'GZMB',
              'GNLY',
              'NKG7',
              'FOXP3',
              'IL2RA',
              'TIGIT',
              'HAVCR2',
              'CTLA4',
              'LAG3',
              'PDCD1',
              'TOX',
              'FCGR3A',
              'KIR3DL2',
              'TYROBP',
              'NCAM1',
              'CD160']

In [ ]:
#Plot T cells subset markers from Zhang 2021 Nature Immuno covid paper blood/balf. 
sc.pl.dotplot(rna_x, Zhang2020_T, groupby = 'leiden_R2.3')

In [ ]:
#Visualise
sc.pl.umap(rna_x, color=['leiden_R2.2', 'Sampletype'], legend_loc='on data')

In [ ]:
#Name clusters based on DEG and published marker gene expression. 
new_dict1 = {'0':'CD4 Tn',
             '1,0':'DOUBLET',
             '1,1':'DOUBLET',
             '1,2':'DOUBLET',
             '2,0':'CD4 Trm',
             '2,1':'CD4 T ICOS+',
             '2,2':'CD4 Trm',
             '2,3':'CD4 Tcm',
             '2,4':'CD4 Tmem AREG+',
             '3,0':'CD8 Trm',
             '3,1':'MAIT',
             '3,2':'DOUBLET',
             '4':'CD8 Tn',
             '5':'DOUBLET',
             '6':'CD4 Tem',
             '7,0':'Tfh',
             '7,1':'Treg',
             '8':'DOUBLET',
             '9':'CD4 Tcm',
             '10':'CD4 Tn',
             '11':'CD4 Tcm',
             '12':'CD4 Tcm',
             '13':'gdT vd2vg9',
             '14':'CD4 Tcm',
             '15':'DOUBLET',
             '16':'CD4 Tcm',
             '17':'CD4 Tcm',
             '18':'CD4 Tcm',
             '19':'CD4 Tcm',
             '20':'CD4 Tcm CXCR4+',
             '21':'CD4 Tcm',
             '22':'CD4 Tcm ',
             '23':'CD4 Tcm',
             '24':'DOUBLET',
             '25':'CD4 Tn',
             '26':'CD4 Tcm',
             '27':'DOUBLET',
            '28':'DOUBLET'}
rna_x.obs['fine_label'] = [new_dict1[l] for l in rna_x.obs['leiden_R2.3']]
sc.pl.umap(rna_x, color=['fine_label'], size=20, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
adata_concat

In [ ]:
# update the original object
adata_concat.obs['fine_label'] = adata_concat.obs['fine_label'].astype('object')
adata_concat.obs['fine_label'].update(rna_x.obs['fine_label'].astype('object'))
sc.pl.umap(adata_concat, color = ['leiden', 'fine_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#save for now
adata_concat.write('COV_combined_T-fine_label_060721.h5ad', compression = 'gzip')
adata_concat.raw.to_adata().write('COV_combined_T_fine_label_raw_060721.h5ad', compression = 'gzip')

In [ ]:
#Visualise
sc.pl.umap(adata_concat, color=['leiden','broad_label','fine_label'], legend_loc = 'on data')

In [ ]:
#need to recluster T cells without doublets, but do this later in next itteration of object. . 

### NK cell

In [ ]:
# subset NK cells
rna_ = adata_concat[adata_concat.obs['leiden'].isin(['3'])]
rna_x = sc.AnnData(X = rna_.raw.X, obs = rna_.obs, var = rna_.raw.var, uns = rna_.uns, obsm = rna_.obsm, obsp = rna_.obsp)
rna_x.raw = rna_x
sc.pp.highly_variable_genes(rna_x, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(rna_x)
rna_x

In [ ]:
## remove TRBV/TRAV and IGHV/IGLV/IGKV from the highly variable genes
## keep TCR gammadelta genes
import re
for i in rna_x.var.index:
    if re.search('^TR[AB]V|^IG[HKL]V', i):
        rna_x.var.at[i, 'highly_variable'] = False
sc.pl.highly_variable_genes(rna_x)

In [ ]:
# subset to highly variable
rna_x = rna_x[:, rna_x.var['highly_variable']]
# regress and scale for PCA
sc.pp.regress_out(rna_x, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(rna_x, max_value = 10)

In [ ]:
# Principal component analysis
sc.tl.pca(rna_x, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(rna_x, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
#Visulise where main object lieden cluster fall
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# run harmony for bathc correction 
sc.external.pp.harmony_integrate(rna_x, 'Patient')
'X_pca_harmony' in rna_x.obsm


In [ ]:
#Check 
rna_x

In [ ]:
# Compute the neighborhood graph for the harmony corrected object. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP of harmony batch corrected object. 
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
# find subset leiden clusters
sc.tl.leiden(rna_x, resolution = 1, key_added = 'leiden_R')
sc.pl.umap(rna_x, color=['leiden', 'leiden_R'], size=6, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Look at NK/GDT/CD8/MAIT marker genes
sc.pl.umap(rna_x, color = ['TRDC', 'TRGV9', 'CD4', 'CD8B', 'FOXP3', 'CXCR5', 'PDCD1', 'CD8A','TRAV1-2', 'CD69', 'KLRB1','ITGAE'], size = 20)

In [ ]:
sc.pl.umap(rna_x, color = ['SELL', 'IFNG', 'PTGDR2','CCR3','CCR8','CXCR3','CCR5','IL13', 'IL5', 'IL4', 'IL17A', 'TRGC2', 'TRDV1', 'CD3E','TRGC1', 'TRDV2'], size = 20)

In [ ]:
sc.pl.umap(rna_x, color = ['FCGR3A', 'NKG7','CD8B','CD3E','NCAM1','KLRC2','RORC','KIT'], size = 20)

In [ ]:
# split specific clusters based on gene expression 
sc.tl.leiden(rna_x, resolution = .3, key_added = 'leiden_R2', restrict_to =('leiden_R', ['1']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split specific clusters
sc.tl.leiden(rna_x, resolution = .4, key_added = 'leiden_R2.1', restrict_to =('leiden_R2', ['2']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2.1'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split specific clusters
sc.tl.leiden(rna_x, resolution = .3, key_added = 'leiden_R2.2', restrict_to =('leiden_R2.1', ['5']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2.2'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Top 10 DEG dotplots 
sc.tl.rank_genes_groups(rna_x, groupby = 'leiden_R2.2', method = 'wilcoxon')
sc.tl.filter_rank_genes_groups(rna_x, min_fold_change=1)
sc.tl.dendrogram(rna_x, groupby = 'leiden_R2.2')
sc.pl.rank_genes_groups_dotplot(rna_x, n_genes = 10, standard_scale = 'var', color_map = 'viridis', key = 'rank_genes_groups_filtered')

In [ ]:
#Chua 2020 gene expression profiles by cluster 
sc.pl.dotplot(rna_x, Chuafeat, groupby = 'leiden_R2.2')

In [ ]:
sc.pl.dotplot(rna_x, Trav_list, groupby = 'leiden_R2.2')

In [ ]:
#Travagliini genen expression profiles by cluster 
sc.pl.dotplot(rna_x, ILC_Bjorkland, groupby = 'leiden_R2.2')

In [ ]:
#Zhang T cell gene expression porfolies by cluster 
sc.pl.dotplot(rna_x, Zhang2020_T, groupby = 'leiden_R2.2')

In [ ]:
#Show split of data by sampletype and leiden cluster. 
sc.pl.umap(rna_x, color=['leiden_R2.2', 'Sampletype'], legend_loc='on data')

In [ ]:
#Name clusters based on gene expression profiles 
new_dict1 = {'0':'CD16+ NK',
             '1,0':'CD8 CTL',
             '1,1':'gdT vd1',
             '2,0':'CD8 Tcm',
             '2,1':'MAIT',
             '2,2':'DOUBLET',
             '3':'CD16+ NK PTGDS+',
             '4':'aNK KLRC2+',
             '5,0':'CD56+ NK',
             '5,1':'CD56+ NK tissue',
             '6':'DOUBLET',
             '7':'CD8 Trm',
             '8':'DOUBLET',
             '9':'DOUBLET',
             '10':'gdT vg3vd3',
             '11':'DOUBLET',
             '12':'ILC',
             '13':'DOUBLET',
             '14':'CD16+NK ISG+'
             }
rna_x.obs['fine_label'] = [new_dict1[l] for l in rna_x.obs['leiden_R2.2']]
sc.pl.umap(rna_x, color=['fine_label'], size=20, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# update the original object with leiden clsuter 3 subset annotations 
adata_concat.obs['fine_label'] = adata_concat.obs['fine_label'].astype('object')
adata_concat.obs['fine_label'].update(rna_x.obs['fine_label'].astype('object'))
sc.pl.umap(adata_concat, color = ['leiden', 'fine_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#save for now
adata_concat.write('COV_combined_TNK-fine_label_060721.h5ad', compression = 'gzip')
adata_concat.raw.to_adata().write('COV_combined_TNK_fine_label_raw_060721.h5ad', compression = 'gzip')

In [ ]:
#Visualise 
sc.pl.umap(adata_concat, color=['leiden','broad_label','fine_label'], legend_loc = 'on data')

### B cells

In [ ]:
# subset B and plasma broad labelled clusters 
rna_ = adata_concat[adata_concat.obs['leiden'].isin(['1', '4', '8','9','21'])]
rna_x = sc.AnnData(X = rna_.raw.X, obs = rna_.obs, var = rna_.raw.var, uns = rna_.uns, obsm = rna_.obsm, obsp = rna_.obsp)
rna_x.raw = rna_x
sc.pp.highly_variable_genes(rna_x, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(rna_x)
rna_x

In [ ]:
## remove TRBV/TRAV and IGHV/IGLV/IGKV from the highly variable genes
import re
for i in rna_x.var.index:
    if re.search('^TR[AB]V|^IG[HKL]V', i):
        rna_x.var.at[i, 'highly_variable'] = False
sc.pl.highly_variable_genes(rna_x)

In [ ]:
# subset to highly variable
rna_x = rna_x[:, rna_x.var['highly_variable']]
# regress and scale for PCA
sc.pp.regress_out(rna_x, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(rna_x, max_value = 10)

In [ ]:
# Principal component analysis
sc.tl.pca(rna_x, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(rna_x, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# run harmony batch correction 
sc.external.pp.harmony_integrate(rna_x, 'Patient')
'X_pca_harmony' in rna_x.obsm


In [ ]:
rna_x

In [ ]:
# Compute the neighborhood graph with harmony batch correction. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP of harmony corrected object 
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
#Visualise how main object clusters are split in reclustered object 
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# find clusters in subsetted object 
sc.tl.leiden(rna_x, resolution = 0.5, key_added = 'leiden_R')
sc.pl.umap(rna_x, color=['leiden', 'leiden_R'], size=6, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split clusters
sc.tl.leiden(rna_x, resolution = .2, key_added = 'leiden_R2', restrict_to = ('leiden_R', ['3']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Explore expression of key B cell/doublet/non B cell genes
sc.pl.umap(rna_x, color = ['CD27', 'ITGA2B', 'CD19', 'CD3D', 'GNLY', 'JCHAIN', 'S100A8', 'CD3E'], size = 20)

In [ ]:
sc.pl.umap(rna_x, color = ['PPBP', 'HBB','CD38'], size = 20)

In [ ]:
#top 10 DEG ranked dotplot by cluster 
sc.tl.rank_genes_groups(rna_x, groupby = 'leiden_R2', method = 'wilcoxon')
sc.tl.filter_rank_genes_groups(rna_x, min_fold_change=1)
sc.tl.dendrogram(rna_x, groupby = 'leiden_R2')
sc.pl.rank_genes_groups_dotplot(rna_x, n_genes = 10, standard_scale = 'var', color_map = 'viridis', key = 'rank_genes_groups_filtered')

In [ ]:
#Chua et al gene expression profiles by cluster 
sc.pl.dotplot(rna_x, Chuafeat, groupby = 'leiden_R2')

In [ ]:
#Travaglini gene expression profiles by cluster 
sc.pl.dotplot(rna_x, Trav_list, groupby = 'leiden_R2.2')

In [ ]:
#Bjorkland ILC genes by cluster 
sc.pl.dotplot(rna_x, ILC_Bjorkland, groupby = 'leiden_R2.2')

In [ ]:
#Zhang t cell profiles by cluster 
sc.pl.dotplot(rna_x, Zhang2020_T, groupby = 'leiden_R2.2')

In [ ]:
#A collection of key B cell and plasma cell markers for different maturation stages 
Bmark = ['CD19', 'MS4A1','CD38','JCHAIN','MME','CD27','TNFRSF13C', 'TNFRSF13B',
         'TNFRSF17','CR2','CD5','FCER2','CD24','IGHD','IGHM','IGHE','IGHA1','IGHG1', 'HLA-DRA', 'CD79A',
        'CXCR4','CXCR5','CD34','CD86','CD1D','CD74']
Bmark

In [ ]:
#B cell markers expressed in a dotplot 
sc.pl.dotplot(rna_x,Bmark,groupby='leiden_R2')

In [ ]:
#Name B cells subset clusters in accordance with gene expression profiles above 
new_dict1 = {'0':'DOUBLET',
             '1':'B',
             '2':'B',
             '3,0':'Plasma IgA',
             '3,1':'DOUBLET',
             '4':'IgM+ Plasmablast',
             '5':'DOUBLET',
             '6':'Non-switched B',
             '7':'Bmem',
             '8':'B',
             '9':'B',
             '10':'Follicular B',
             '11':'Non-switched B',
             '12':'Transitional B',
             '13':'Transitional B',
             '14':'B',
             '15':'DOUBLET',
             '16':'Marginal zone B',
             '17':'B',
             '18':'Plasmablast',
             '19':'Bmem BHLHE40+',
             '20':'B',
             '21':'B',
             '22':'B',
             '23':'B IFNstim GBP5+',
             '24':'B',
             '25':'B',
             '26':'DOUBLET',
             '27':'B',
             '28':'Bn'
}
rna_x.obs['fine_label'] = [new_dict1[l] for l in rna_x.obs['leiden_R2']]
sc.pl.umap(rna_x, color=['fine_label'], size=20, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# update the original object with B cell subset annotations 
adata_concat.obs['fine_label'] = adata_concat.obs['fine_label'].astype('object')
adata_concat.obs['fine_label'].update(rna_x.obs['fine_label'].astype('object'))
sc.pl.umap(adata_concat, color = ['leiden', 'fine_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#save for now
adata_concat.write('COV_combined_TNKB-fine_label_060721.h5ad', compression = 'gzip')
adata_concat.raw.to_adata().write('COV_combined_TNKB_fine_label_raw_060721.h5ad', compression = 'gzip')

In [ ]:
#load as kernel died
adata_concat = sc.read_h5ad('COV_combined_TNKB-fine_label_060721.h5ad')
adata_concat

In [ ]:
#Visualise
sc.pl.umap(adata_concat, color = ['leiden', 'broad_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

### monocyte/macrophage/DC

In [ ]:
# subset MMP/DC broad labelled clusters. 
rna_ = adata_concat[adata_concat.obs['leiden'].isin(['2'])]
rna_x = sc.AnnData(X = rna_.raw.X, obs = rna_.obs, var = rna_.raw.var, uns = rna_.uns, obsm = rna_.obsm, obsp = rna_.obsp)
rna_x.raw = rna_x
sc.pp.highly_variable_genes(rna_x, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(rna_x)
rna_x

In [ ]:
# subset to highly variable
rna_x = rna_x[:, rna_x.var['highly_variable']]
# regress and scale for PCA
sc.pp.regress_out(rna_x, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(rna_x, max_value = 10)

In [ ]:
# Principal component analysis
sc.tl.pca(rna_x, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(rna_x, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# run harmony for batch correction 
sc.external.pp.harmony_integrate(rna_x, 'Patient')
'X_pca_harmony' in rna_x.obsm


In [ ]:
rna_x

In [ ]:
# Compute the neighborhood graph with harmony correction. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP with harmony correction 
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# find clusters in subsetted object. 
sc.tl.leiden(rna_x, resolution = 1, key_added = 'leiden_R')
sc.pl.umap(rna_x, color=['leiden', 'leiden_R'], size=6, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split clusters
sc.tl.leiden(rna_x, resolution = .2, key_added = 'leiden_R2', restrict_to = ('leiden_R', ['6']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Review expression of key MMP/DC/Doublet marker genes. 
sc.pl.umap(rna_x, color = ['CD14', 'FCGR3A', 'FCGR3B', 'CD68', 'GNLY', 'CD3E', 'CD1C', 'CLEC9A', 'C1QC', 'CD86', 'PPBP', 'HBB'], size = 20)

In [ ]:
#Top DEG dotplot by cluster 
sc.tl.rank_genes_groups(rna_x, groupby = 'leiden_R2', method = 'wilcoxon')
sc.tl.filter_rank_genes_groups(rna_x, min_fold_change=1)
sc.tl.dendrogram(rna_x, groupby = 'leiden_R2')
sc.pl.rank_genes_groups_dotplot(rna_x, n_genes = 10, standard_scale = 'var', color_map = 'viridis', key = 'rank_genes_groups_filtered')

In [ ]:
#Chau marker gene expression by cluster 
sc.pl.dotplot(rna_x, Chuafeat, groupby = 'leiden_R2')

In [ ]:
#Travalgini marker gene expression by cluster 
sc.pl.dotplot(rna_x, Trav_list, groupby = 'leiden_R2')

In [ ]:
#Bjorkland ILC geneset expression 
sc.pl.dotplot(rna_x, ILC_Bjorkland, groupby = 'leiden_R2')

In [ ]:
#Zhang T cell marker gene expression by cluster 
sc.pl.dotplot(rna_x, Zhang2020_T, groupby = 'leiden_R2')

In [ ]:
#Visualise clusters 
sc.pl.umap(rna_x, color = 'leiden_R2', legend_loc = 'on data')

In [ ]:
#Label clusters by gene expression profiles 
new_dict1 = {'0':'DOUBLET',
             '1':'Classical monocyte',
             '2':'Classical monocyte',
             '3':'DOUBLET',
             '4':'Classical monocyte',
             '5':'Macrophage',
             '6,0':'DOUBLET',
             '6,1':'DOUBLET',
             '6,2':'DOUBLET',
             '7':'DOUBLET',
             '8':'Classical monocyte',
             '9':'cDC CD1C+',
             '10':'DOUBLET',
             '11':'DOUBLET',
             '12':'DOUBLET',
             '13':'DOUBLET',
             '14':'DOUBLET',
             '15':'Non-classical monocyte',
             '16':'DOUBLET',
             '17':'Basophil'
}
rna_x.obs['fine_label'] = [new_dict1[l] for l in rna_x.obs['leiden_R2']]
sc.pl.umap(rna_x, color=['fine_label'], size=20, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# update the original object
adata_concat.obs['fine_label'] = adata_concat.obs['fine_label'].astype('object')
adata_concat.obs['fine_label'].update(rna_x.obs['fine_label'].astype('object'))
sc.pl.umap(adata_concat, color = ['leiden', 'fine_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#Visualise 
sc.pl.umap(adata_concat, color = ['leiden', 'broad_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#save for now
adata_concat.write('COV_combined_TNKBMMP-fine_label_070721.h5ad', compression = 'gzip')
adata_concat.raw.to_adata().write('COV_combined_TNKBMMP_fine_label_raw_070721.h5ad', compression = 'gzip')

### Epithelial

In [ ]:
# subset broad label epithelial clusters 
rna_ = adata_concat[adata_concat.obs['leiden'].isin(['5','10','29','30'])]
rna_x = sc.AnnData(X = rna_.raw.X, obs = rna_.obs, var = rna_.raw.var, uns = rna_.uns, obsm = rna_.obsm, obsp = rna_.obsp)
rna_x.raw = rna_x
sc.pp.highly_variable_genes(rna_x, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(rna_x)
rna_x

In [ ]:
# subset to highly variable
rna_x = rna_x[:, rna_x.var['highly_variable']]
# regress and scale for PCA
sc.pp.regress_out(rna_x, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(rna_x, max_value = 10)

In [ ]:
# Principal component analysis
sc.tl.pca(rna_x, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(rna_x, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# run harmony for bathc correction 
sc.external.pp.harmony_integrate(rna_x, 'Patient')
'X_pca_harmony' in rna_x.obsm


In [ ]:
rna_x

In [ ]:
# Compute the neighborhood graph with harmony correction. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP with harmony correction 
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# find clusters in subet object 
sc.tl.leiden(rna_x, resolution = 0.8, key_added = 'leiden_R')
sc.pl.umap(rna_x, color=['leiden', 'leiden_R'], size=6, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split clusters
sc.tl.leiden(rna_x, resolution = .2, key_added = 'leiden_R2', restrict_to = ('leiden_R', ['5']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# split clusters
sc.tl.leiden(rna_x, resolution = .2, key_added = 'leiden_R2.1', restrict_to = ('leiden_R2', ['4']))
sc.pl.umap(rna_x, color=['leiden', 'leiden_R2.1'], size=10, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Explore expression of key epithelial marker genes 
sc.pl.umap(rna_x, color = ['EPCAM','KRT19','MUC5AC','PRR4','COL1A1','ACTA2','FOXJ1','ACKR1','PPBP', 'HBB','CD1A','ASCL2'], size = 20)

In [ ]:
#Top 10 DEG by cluster 
sc.tl.rank_genes_groups(rna_x, groupby = 'leiden_R2.1', method = 'wilcoxon')
sc.tl.filter_rank_genes_groups(rna_x, min_fold_change=1)
sc.tl.dendrogram(rna_x, groupby = 'leiden_R2.1')
sc.pl.rank_genes_groups_dotplot(rna_x, n_genes = 10, standard_scale = 'var', color_map = 'viridis', key = 'rank_genes_groups_filtered')

In [ ]:
#Chua marker gene expression profiles by cluster
sc.pl.dotplot(rna_x, Chuafeat, groupby = 'leiden_R2.1')

In [ ]:
#Travaglini marker gene expression profiles by cluster
sc.pl.dotplot(rna_x, Trav_list, groupby = 'leiden_R2.1')

In [ ]:
#Visualise clusters 
sc.pl.umap(rna_x, color = 'leiden_R2.1', legend_loc = 'on data')

In [ ]:
#Name clusters based on gene expression profiles 
new_dict1 = {'0':'DOUBLET',
             '1':'Secretory cell',
             '2':'DOUBLET',
             '3':'DOUBLET',
             '4,0':'Ciliated cell',
             '4,1':'Ciliated cell',
             '4,2':'Ciliated cell',
             '4,3':'Ciliated cell',
             '5,0':'Suprabasal cell',
             '5,1':'Basal cell',
             '5,2':'Squamous epithelium',
             '5,3':'DOUBLET',
             '6':'DOUBLET',
             '7':'DOUBLET',
             '8':'DOUBLET',
             '9':'DOUBLET',
             '10':'DOUBLET',
             '11':'Follicular DC',
             '12':'Secretory cell',
             '13':'DOUBLET',
             '14':'DOUBLET',
             '15':'DOUBLET',
             '16':'Secretory cell IFITM3+',
             '17':'Epithelial cell',
             '18':'Secretory cell',
             '19':'DOUBLET',
             '20':'Endothelial cell'
             
             
}
rna_x.obs['fine_label'] = [new_dict1[l] for l in rna_x.obs['leiden_R2.1']]
sc.pl.umap(rna_x, color=['fine_label'], size=20, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# update the original object
adata_concat.obs['fine_label'] = adata_concat.obs['fine_label'].astype('object')
adata_concat.obs['fine_label'].update(rna_x.obs['fine_label'].astype('object'))
sc.pl.umap(adata_concat, color = ['leiden', 'fine_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#Visualise
sc.pl.umap(adata_concat, color = ['leiden', 'broad_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#save for now
adata_concat.write('COV_combined_TNKBMMPepi-fine_label_070721.h5ad', compression = 'gzip')
adata_concat.raw.to_adata().write('COV_combined_TNKBMMPepi_fine_label_raw_070721.h5ad', compression = 'gzip')

### pDC

In [ ]:
# subset cells with pDC broad label 
rna_ = adata_concat[adata_concat.obs['leiden'].isin(['22'])]
rna_x = sc.AnnData(X = rna_.raw.X, obs = rna_.obs, var = rna_.raw.var, uns = rna_.uns, obsm = rna_.obsm, obsp = rna_.obsp)
rna_x.raw = rna_x
sc.pp.highly_variable_genes(rna_x, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(rna_x)
rna_x

In [ ]:
# subset to highly variable
rna_x = rna_x[:, rna_x.var['highly_variable']]
# regress and scale for PCA
sc.pp.regress_out(rna_x, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(rna_x, max_value = 10)

In [ ]:
# Principal component analysis
sc.tl.pca(rna_x, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(rna_x, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# run harmony batch correction 
sc.external.pp.harmony_integrate(rna_x, 'Patient')
'X_pca_harmony' in rna_x.obsm


In [ ]:
rna_x

In [ ]:
# Compute the neighborhood graph with harmony batch correction . Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP with harmony batch correction 
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# find clusters in subset object 
sc.tl.leiden(rna_x, resolution = 0.5, key_added = 'leiden_R')
sc.pl.umap(rna_x, color=['leiden', 'leiden_R'], size=6, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Some DC/pDC related marker gene expression 
sc.pl.umap(rna_x, color = ['IL3RA','CLEC10A','THBD'], size = 20)

In [ ]:
#Top 10 DEG by cluster 
sc.tl.rank_genes_groups(rna_x, groupby = 'leiden_R', method = 'wilcoxon')
sc.tl.filter_rank_genes_groups(rna_x, min_fold_change=1)
sc.tl.dendrogram(rna_x, groupby = 'leiden_R')
sc.pl.rank_genes_groups_dotplot(rna_x, n_genes = 10, standard_scale = 'var', color_map = 'viridis', key = 'rank_genes_groups_filtered')

In [ ]:
#Chua marker gene expression by cluster 
sc.pl.dotplot(rna_x, Chuafeat, groupby = 'leiden_R')

In [ ]:
#Travaglini marker gene expression by cluster 
sc.pl.dotplot(rna_x, Trav_list, groupby = 'leiden_R')

In [ ]:
#Bjorkland ILC marker genes by cluster 
sc.pl.dotplot(rna_x, ILC_Bjorkland, groupby = 'leiden_R')

In [ ]:
#Zhang t cell marker genes by cluster 
sc.pl.dotplot(rna_x, Zhang2020_T, groupby = 'leiden_R')

In [ ]:
#Visualise
sc.pl.umap(rna_x, color = 'leiden_R', legend_loc = 'on data')

In [ ]:
#Rename subset clusters based on gene expression profiles 
new_dict1 = {'0':'DOUBLET',
             '1':'pDC',
             '2':'pDC',
             '3':'DOUBLET',
             '4':'DOUBLET',
             '5':'DOUBLET',
             '6':'DOUBLET',
             '7':'pDC',
             '8':'DOUBLET',
             '9':'pDC',
             '10':'pDC',
             '11':'pDC',
             '12':'pDC',
             '13':'pDC',
             '14':'ILC',
             '15':'pDC',
             '16':'DOUBLET',
             '17':'DOUBLET',
             '18':'pDC'
             
             
}
rna_x.obs['fine_label'] = [new_dict1[l] for l in rna_x.obs['leiden_R']]
sc.pl.umap(rna_x, color=['fine_label'], size=20, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# update the original object
adata_concat.obs['fine_label'] = adata_concat.obs['fine_label'].astype('object')
adata_concat.obs['fine_label'].update(rna_x.obs['fine_label'].astype('object'))
sc.pl.umap(adata_concat, color = ['leiden', 'fine_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#Visualise 
sc.pl.umap(adata_concat, color = ['leiden', 'broad_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#save for now
adata_concat.write('COV_combined_TNKBMMPepipDC-fine_label_070721.h5ad', compression = 'gzip')
adata_concat.raw.to_adata().write('COV_combined_TNKBMMPepipDC_fine_label_raw_070721.h5ad', compression = 'gzip')

### Megakaryocyte and erythrocyte

In [ ]:
# subset broad labelled megakaryocytes and red cells 
rna_ = adata_concat[adata_concat.obs['leiden'].isin(['15', '16'])]
rna_x = sc.AnnData(X = rna_.raw.X, obs = rna_.obs, var = rna_.raw.var, uns = rna_.uns, obsm = rna_.obsm, obsp = rna_.obsp)
rna_x.raw = rna_x
sc.pp.highly_variable_genes(rna_x, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(rna_x)
rna_x

In [ ]:
# subset to highly variable
rna_x = rna_x[:, rna_x.var['highly_variable']]
# regress and scale for PCA
sc.pp.regress_out(rna_x, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(rna_x, max_value = 10)

In [ ]:
# Principal component analysis
sc.tl.pca(rna_x, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(rna_x, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
#What is broad leiden cluster split?
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# run harmony for batch correction 
sc.external.pp.harmony_integrate(rna_x, 'Patient')
'X_pca_harmony' in rna_x.obsm


In [ ]:
rna_x

In [ ]:
# Compute the neighborhood graph with harmony correction . Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP with harmony correction 
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
#Broad leiden cluster split 
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# find subset clusters
sc.tl.leiden(rna_x, resolution = 0.5, key_added = 'leiden_R')
sc.pl.umap(rna_x, color=['leiden', 'leiden_R'], size=6, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Marker genes of key celltypes given high likelihood of doublets
sc.pl.umap(rna_x, color = ['CD14', 'FCGR3A', 'FCGR3B', 'CD68', 'GNLY', 'CD3E', 'CD1C', 'CLEC9A', 'C1QC', 'CD86', 'PPBP', 'HBB'], size = 20)

In [ ]:
#Top 10 DEG by cluster 
sc.tl.rank_genes_groups(rna_x, groupby = 'leiden_R', method = 'wilcoxon')
sc.tl.filter_rank_genes_groups(rna_x, min_fold_change=1)
sc.tl.dendrogram(rna_x, groupby = 'leiden_R')
sc.pl.rank_genes_groups_dotplot(rna_x, n_genes = 10, standard_scale = 'var', color_map = 'viridis', key = 'rank_genes_groups_filtered')

In [ ]:
#Chua marker genes by cluster 
sc.pl.dotplot(rna_x,Chuafeat,groupby ='leiden_R')

In [ ]:
#Name subset clusters based on gene expression profiles 
new_dict1 = {'0':'DOUBLET',
             '1':'DOUBLET',
            '2':'Megakaryocyte',
             '3':'DOUBLET',
             '4':'DOUBLET',
             '5':'DOUBLET',
             '6':'DOUBLET',
             '7':'DOUBLET',
            '8':'Erythrocyte',
             '9':'DOUBLET',
             '10':'DOUBLET',
             '11':'DOUBLET',
             '12':'DOUBLET',
             '13':'DOUBLET',
             '14':'DOUBLET',
             '15':'DOUBLET',
             '16':'DOUBLET',
             '17':'DOUBLET',
            '18':'Basophil'}
rna_x.obs['fine_label'] = [new_dict1[l] for l in rna_x.obs['leiden_R']]
sc.pl.umap(rna_x, color=['fine_label'], size=20, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# update the original object
adata_concat.obs['fine_label'] = adata_concat.obs['fine_label'].astype('object')
adata_concat.obs['fine_label'].update(rna_x.obs['fine_label'].astype('object'))
sc.pl.umap(adata_concat, color = ['leiden', 'fine_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

### Mast

In [ ]:
# subset Mast cell broad cluster 
rna_ = adata_concat[adata_concat.obs['leiden'].isin(['25'])]
rna_x = sc.AnnData(X = rna_.raw.X, obs = rna_.obs, var = rna_.raw.var, uns = rna_.uns, obsm = rna_.obsm, obsp = rna_.obsp)
rna_x.raw = rna_x
sc.pp.highly_variable_genes(rna_x, min_mean = 0.0125, max_mean = 3, min_disp = 0.5)
sc.pl.highly_variable_genes(rna_x)
rna_x

In [ ]:
# subset to highly variable
rna_x = rna_x[:, rna_x.var['highly_variable']]
# regress and scale for PCA
sc.pp.regress_out(rna_x, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(rna_x, max_value = 10)

In [ ]:
# Principal component analysis
sc.tl.pca(rna_x, svd_solver = 'arpack')
sc.pl.pca_variance_ratio(rna_x, log = True, n_pcs = 50)

In [ ]:
# Computing the neighborhood graph. Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50)

In [ ]:
# run UMAP
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# run harmony for batch correction
sc.external.pp.harmony_integrate(rna_x, 'Patient')
'X_pca_harmony' in rna_x.obsm


In [ ]:
rna_x

In [ ]:
# Compute the neighborhood graph based on harmony correction (not didn't converge). Seurat uses k = 20 as default
sc.pp.neighbors(rna_x, n_neighbors = 10, n_pcs = 50, use_rep='X_pca_harmony')

In [ ]:
# UMAP based on harmoyn correction
sc.tl.umap(rna_x, n_components = 2, min_dist = 0.3)
sc.pl.umap(rna_x, color=['Sampleid','Patient', 'Sampletype'])

In [ ]:
sc.pl.umap(rna_x, color=['leiden'])

In [ ]:
# find clusters in subset object 
sc.tl.leiden(rna_x, resolution = 0.5, key_added = 'leiden_R')
sc.pl.umap(rna_x, color=['leiden', 'leiden_R'], size=6, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
#Explore gene expression in clusters 
sc.pl.umap(rna_x, color = ['CD14', 'FCGR3A','GATA2','MRC1','GATA3', 'CD3E', 'CD1C', 'CLEC9A', 'C1QC', 'CD86', 'PPBP', 'HBB'], size = 20)

In [ ]:
#Top 10 DEG by cluster 
sc.tl.rank_genes_groups(rna_x, groupby = 'leiden_R', method = 'wilcoxon')
sc.tl.filter_rank_genes_groups(rna_x, min_fold_change=1)
sc.tl.dendrogram(rna_x, groupby = 'leiden_R')
sc.pl.rank_genes_groups_dotplot(rna_x, n_genes = 10, standard_scale = 'var', color_map = 'viridis', key = 'rank_genes_groups_filtered')

In [ ]:
#Chua marker gene expression profiles by cluster 
sc.pl.dotplot(rna_x,Chuafeat,groupby ='leiden_R')

In [ ]:
#Bjorkland ILC marker gene expression profiles by cluster 
sc.pl.dotplot(rna_x,ILC_Bjorkland,groupby ='leiden_R')

In [ ]:
#Rename clusters based on gene expression profiles. 
new_dict1 = {'0':'DOUBLET',
             '1':'Macrophage',
             '2':'Mast',
             '3':'M2 Macrophage',
             '4':'DOUBLET',
             '5':'Macrophage',
             '6':'DOUBLET',
             '7':'DOUBLET',
             '8':'DOUBLET',
             '9':'DOUBLET',
             '10':'Macrophage',
             '11':'Macrophage',
             '12':'DOUBLET',
             '13':'DOUBLET',
             '14':'DOUBLET',
             '15':'Macrophage',
             '16':'DOUBLET',
            '17':'Mast',
            '18':'DOUBLET',
            '19':'DOUBLET',
            '20':'DOUBLET',
            '21':'DOUBLET'}
rna_x.obs['fine_label'] = [new_dict1[l] for l in rna_x.obs['leiden_R']]
sc.pl.umap(rna_x, color=['fine_label'], size=20, legend_loc ='on data', legend_fontoutline=2)

In [ ]:
# update the original object
adata_concat.obs['fine_label'] = adata_concat.obs['fine_label'].astype('object')
adata_concat.obs['fine_label'].update(rna_x.obs['fine_label'].astype('object'))
sc.pl.umap(adata_concat, color = ['leiden', 'fine_label'], size = 10, legend_loc = 'on data', legend_fontoutline=2)

In [ ]:
#save fully annotated first itteration (for reloading in new notebook for removal of doublets and reclustering)
adata_concat.write('COV_combined_all_fine_label_070721.h5ad', compression = 'gzip')
adata_concat.raw.to_adata().write('COV_combined_all_fine_label_raw_070721.h5ad', compression = 'gzip')